In [15]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [16]:
TXN_FILE_PATH = "https://websdk-assets.s3.ap-south-1.amazonaws.com/public/txns+(13).csv"
REPAY_FILE_PATH = "https://websdk-assets.s3.ap-south-1.amazonaws.com/public/repayments+(2).csv"

In [17]:
TXN_DF = pd.read_csv(TXN_FILE_PATH, parse_dates=['day'])
REPAY_DF = pd.read_csv(REPAY_FILE_PATH, parse_dates=['repaymentDay'])

### Query 1

In [18]:
def calculate_total_outstanding():
    # Calculate the total principal to be paid for each loan (leaving out interest as per the query's description)
    principal_to_pay_series = TXN_DF['downPaymentAmount'] + (TXN_DF['tenure'] * TXN_DF['emiPrincipal'])
    principal_to_pay_df = pd.DataFrame({'transactionId' : TXN_DF['transactionId'], 'totalPrincipalToPay' : principal_to_pay_series})

    # Group repayments by transactionId and sum the repayments towards each individial loan
    total_repayments_df = REPAY_DF.groupby('transactionId')['amount'].sum().reset_index()

    # Merge transactions with total repayments
    paid_unpaid_df = pd.merge(principal_to_pay_df, total_repayments_df, on='transactionId', how='left')
    paid_unpaid_df.rename(columns={'amount': 'totalPrincipalPaid'}, inplace=True)

    # Replace NaN with 0 for loans with no repayments (not the case in this dataset, but adding for future)
    paid_unpaid_df['totalPrincipalPaid'] = paid_unpaid_df['totalPrincipalPaid'].fillna(0)

    # Calculate outstanding amount to be paid
    paid_unpaid_df['outstandingAmount'] = (paid_unpaid_df['totalPrincipalToPay'] - paid_unpaid_df['totalPrincipalPaid'])

    # Select required columns
    final_result = pd.DataFrame(paid_unpaid_df[['transactionId', 'outstandingAmount']])
    # Convert paise to rupees
    final_result['outstandingAmount'] = final_result['outstandingAmount'] / 100
    return final_result


In [19]:
calculate_total_outstanding()

,transactionId,outstandingAmount
0,704663,0.00
1,1403240,0.00
2,1961345,0.00
3,1682956,0.00
4,2392323,2811.34
...,...,...
995,1076397,0.00
996,945211,0.00
997,541382,0.00
998,336716,0.00


### Query 2

In [20]:
def calculate_aum(date_str):
    txn_df = pd.DataFrame(TXN_DF)
    repay_df = pd.DataFrame(REPAY_DF)
    
    # Convert input date string to UTC-aware datetime object
    as_of_date = datetime.strptime(date_str, '%Y-%m-%d').replace(tzinfo=pytz.UTC)

    # Making datetime object in dataframes UTC-aware
    txn_df['day'] = txn_df['day'].dt.tz_localize(None).dt.tz_localize('UTC')
    repay_df['repaymentDay'] = repay_df['repaymentDay'].dt.tz_localize(None).dt.tz_localize('UTC')

    # Filter transactions up to the specified date
    txn_df = txn_df[txn_df['day'] <= as_of_date]

    # Calculate total loan amount for each transaction
    txn_df['totalPrincipalToPay'] = txn_df['downPaymentAmount'] + (txn_df['tenure'] * txn_df['emiPrincipal'])

    # Filter repayments up to the specified date
    repay_df = repay_df[repay_df['repaymentDay'] <= as_of_date]

    # Group repayments by transactionId and sum the amounts
    total_repayments = repay_df.groupby('transactionId')['amount'].sum().reset_index()

    # Merge transactions with total repayments (left join)
    result = pd.merge(txn_df[['transactionId', 'totalPrincipalToPay']], total_repayments, on='transactionId', how='left')

    # Replace NaN with 0 for loans with no repayments
    result['amount'] = result['amount'].fillna(0)

    # Calculate outstanding amount
    result['outstandingAmount'] = (result['totalPrincipalToPay'] - result['amount'])
    # Convert paise to rupees
    result['outstandingAmount'] = result['outstandingAmount'] / 100

    # AUM = sum of all outstanding amounts
    aum = result['outstandingAmount'].sum()

    return aum

In [21]:
date = '2022-03-26'
calculate_aum(date)

6266.33

### Query 3

Helper Functions

In [22]:
# Get EMI due dates
def generate_emi_dates(row):
    return [row['day'] + timedelta(days=30*i) for i in range(row['tenure'])]

# Categorize DPD into buckets
def dpd_bucket(dpd):
    if dpd < 0:
        return 'amount_not_due'
    elif 0 <= dpd < 30:
        return 'amount_dpd_0_30'
    elif 30 <= dpd < 60:
        return 'amount_dpd_30_60'
    elif 60 <= dpd < 90:
        return 'amount_dpd_60_90'
    else:
        return 'amount_dpd_90_above'

In [39]:
def calculate_portfolio_performance(as_of_date_str):
    # Convert input date string to datetime without timezone
    as_of_date = pd.to_datetime(as_of_date_str)

    # Read the CSV files
    txn_df = pd.DataFrame(TXN_DF)
    repay_df = pd.DataFrame(REPAY_DF)

    # Ensure datetime columns are timezone-naive
    txn_df['day'] = txn_df['day'].dt.tz_localize(None)
    repay_df['repaymentDay'] = repay_df['repaymentDay'].dt.tz_localize(None)

    # Create cohort column (year-month)
    txn_df['cohort'] = txn_df['day'].dt.to_period('M')

    # Generate EMI due dates for each transaction
    txn_df['emi_dates'] = txn_df.apply(generate_emi_dates, axis=1)

    # Explode the DataFrame so each EMI has its own row
    transactions_exploded = txn_df.explode('emi_dates')

    # Calculate total repayments for each transaction
    total_repayments = repay_df.groupby('transactionId')['amount'].sum().reset_index()

    # Merge total repayments with transactions
    transactions_exploded = pd.merge(transactions_exploded, total_repayments, on='transactionId', how='left')
    transactions_exploded['amount'] = transactions_exploded['amount'].fillna(0)

    # Calculate outstanding amount for each EMI
    transactions_exploded['emi_amount'] = transactions_exploded['emiPrincipal'] + transactions_exploded['emiInterest']
    transactions_exploded['cumulative_emi'] = transactions_exploded.groupby('transactionId').cumcount() + 1
    transactions_exploded['amount_paid'] = transactions_exploded.apply(lambda row: min(row['amount'], row['cumulative_emi'] * row['emi_amount']), axis=1)
    transactions_exploded['outstanding'] = (transactions_exploded['emi_amount'] * transactions_exploded['cumulative_emi']) - transactions_exploded['amount_paid']

    # Calculate DPD
    transactions_exploded['dpd'] = (as_of_date - transactions_exploded['emi_dates']).dt.days

    # Categorize DPD into buckets
    transactions_exploded['dpd_bucket'] = transactions_exploded['dpd'].apply(dpd_bucket)

    # Group by cohort and DPD bucket, sum outstanding amounts
    result = transactions_exploded.groupby(['cohort', 'dpd_bucket'])['outstanding'].sum().unstack(fill_value=0) / 100  # Convert paise to rupees

    # Index by month
    result = result.rename_axis('month').reset_index()

    # Ensure all required columns are present and in the correct order
    required_columns = ['month', 'amount_not_due', 'amount_dpd_0_30', 'amount_dpd_30_60', 'amount_dpd_60_90', 'amount_dpd_90_above']
    result = result.reindex(columns=required_columns, fill_value=0)

    return result


In [40]:
calculate_portfolio_performance('2024-06-24')

dpd_bucket,month,amount_not_due,amount_dpd_0_30,amount_dpd_30_60,amount_dpd_60_90,amount_dpd_90_above
0,2022-02,0.00,0.00,0.00,0.00,0.00
1,2022-03,0.00,0.00,0.00,0.00,0.00
2,2022-04,0.00,0.00,0.00,0.00,4599.97
3,2022-05,0.00,0.00,0.00,0.00,3333.32
4,2022-06,0.00,0.00,0.00,0.00,7000.00
5,2022-07,0.00,0.00,0.00,0.00,2789.52
6,2022-08,0.00,0.00,0.00,0.00,946.82
7,2022-09,0.00,0.00,0.00,0.00,1333.32
8,2022-10,0.00,0.00,0.00,0.00,1299.30
9,2022-11,0.00,0.00,0.00,0.00,1099.65


In [38]:
calculate_portfolio_performance('2024-06-30')

dpd_bucket,month,amount_not_due,amount_dpd_0_30,amount_dpd_30_60,amount_dpd_60_90,amount_dpd_90_above
0,2022-02,0.00,0.00,0.00,0.00,0.00
1,2022-03,0.00,0.00,0.00,0.00,0.00
2,2022-04,0.00,0.00,0.00,0.00,4599.97
3,2022-05,0.00,0.00,0.00,0.00,3333.32
4,2022-06,0.00,0.00,0.00,0.00,7000.00
5,2022-07,0.00,0.00,0.00,0.00,2789.52
6,2022-08,0.00,0.00,0.00,0.00,946.82
7,2022-09,0.00,0.00,0.00,0.00,1333.32
8,2022-10,0.00,0.00,0.00,0.00,1299.30
9,2022-11,0.00,0.00,0.00,0.00,1099.65
